In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType, StringType

In [10]:
spark = SparkSession.builder.appName("ExplodeExample").getOrCreate()

In [11]:
data = [
    (1, "Alice", [{"order_id": 101, "product": "Laptop"}, {"order_id": 102, "product": "Mouse"}]),
    (2, "Bob", [{"order_id": 103, "product": "Keyboard"}, {"order_id": 104, "product": "Monitor"}, {"order_id": 105, "product": "USB Hub"}])
]

In [12]:
order_schema = ArrayType(
    StructType([
        StructField("order_id", IntegerType(), True),
        StructField("product", StringType(), True)
    ])
)

In [13]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("orders", order_schema, True)
])

In [14]:
df = spark.createDataFrame(data, schema)

In [15]:
df.show(truncate=False)

+---+-----+-------------------------------------------------+
|id |name |orders                                           |
+---+-----+-------------------------------------------------+
|1  |Alice|[{101, Laptop}, {102, Mouse}]                    |
|2  |Bob  |[{103, Keyboard}, {104, Monitor}, {105, USB Hub}]|
+---+-----+-------------------------------------------------+



In [23]:
exploded_df = df.withColumn("orders", explode(col("orders")))

In [22]:
exploded_df.show()

+---+-----+--------------------+---------------+
| id| name|              orders|          order|
+---+-----+--------------------+---------------+
|  1|Alice|[{101, Laptop}, {...|  {101, Laptop}|
|  1|Alice|[{101, Laptop}, {...|   {102, Mouse}|
|  2|  Bob|[{103, Keyboard},...|{103, Keyboard}|
|  2|  Bob|[{103, Keyboard},...| {104, Monitor}|
|  2|  Bob|[{103, Keyboard},...| {105, USB Hub}|
+---+-----+--------------------+---------------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `order`.`order_id` cannot be resolved. Did you mean one of the following? [`orders`, `id`, `name`].;
'Project [id#0, name#1, 'order.order_id AS order_id#60, 'order.product AS product#61]
+- Project [id#0, name#1, orders#43]
   +- Generate explode(orders#2), false, [orders#43]
      +- LogicalRDD [id#0, name#1, orders#2], false
